# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# from azureml.core.model import InferenceConfig
# from azureml.core import Model
# from azureml.core import Environment, Experiment, Workspace
# from azureml.widgets import RunDetails
# from azureml.train.sklearn import SKLearn
# from azureml.train.hyperdrive.run import PrimaryMetricGoal
# from azureml.train.hyperdrive.policy import BanditPolicy
# from azureml.train.hyperdrive.sampling import RandomParameterSampling
# from azureml.train.hyperdrive.runconfig import HyperDriveConfig
# from azureml.train.hyperdrive.parameter_expressions import uniform, choice
# from azureml.train.hyperdrive.parameter_expressions import choice, loguniform, uniform
# from azureml.core.dataset import Dataset
# from azureml.core.webservice import AciWebservice, Webservice

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

This dataset contains numerical input variables that come from and PCA algorithm. Due to confidential concerns, the original variables are not available. Features V1, V2, ..., V8 are the outputs of the principal components from the PCA algorithm. The time and amount are the variables that are not transformed by PCA. The target variable is the feature 'Class' and it takes values 1 in case of fraud and 0 otherwise.


This dataset contains transations made by credit cards in September 2012 by European cardholders. This dataset contains 492 frauds out of 284807 transations, and it is highly unbalanced, the positive class (fraud) account for 0.172% of all the transations.


In [2]:
# ws = Workspace.from_config()

# # choose a name for experiment
# experiment_name = 'telco-customer-churn'
# experiment=Experiment(ws, experiment_name)

# # Try to load the dataset from the Workspace. Otherwise, create it from the file
# # NOTE: update the key to match the dataset name
# found = False
# key = "Customer Churn"
# description_text = "Customer Churn DataSet for Udacity Capstone Project"

# if key in ws.datasets.keys(): 
#         found = True
#         dataset = ws.datasets[key] 

# if not found:
#         # Create AML Dataset and register it into Workspace
#         example_data = 'https://raw.githubusercontent.com/srees1988/predict-churn-py/main/customer_churn_data.csv'
#         dataset = Dataset.Tabular.from_delimited_files(example_data)        
#         #Register Dataset in Workspace
#         dataset = dataset.register(workspace=ws,
#                                    name=key,
#                                    description=description_text)


# df = dataset.to_pandas_dataframe()

df = pd.read_csv(
    "https://raw.githubusercontent.com/srees1988/predict-churn-py/main/customer_churn_data.csv"
)

df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Preprocessing data

In [4]:
def clean_data(df):
    df.drop("customerID", axis=1, inplace=True)

    # replace blanks with np.nan
    df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)
    # convert to float64
    df["TotalCharges"] = df["TotalCharges"].astype("float64")

    df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].median())

    # Replace binary values
    df["gender"] = df["gender"].apply(lambda s: 1 if s == "Female" else 0)
    df["Partner"] = df["Partner"].apply(lambda s: 1 if s == "Yes" else 0)
    df["Dependents"] = df["Dependents"].apply(lambda s: 1 if s == "Yes" else 0)
    df["PhoneService"] = df["PhoneService"].apply(lambda s: 1 if s == "Yes" else 0)
    df["PaperlessBilling"] = df["PaperlessBilling"].apply(
        lambda s: 1 if s == "Yes" else 0
    )
    df["Churn"] = df["Churn"].apply(lambda s: 1 if s == "Yes" else 0)

    MultipleLines = pd.get_dummies(df["MultipleLines"], prefix="MultipleLines")
    df.drop("MultipleLines", inplace=True, axis=1)
    df = df.join(MultipleLines)
    InternetService = pd.get_dummies(df["InternetService"], prefix="InternetService")
    df.drop("InternetService", inplace=True, axis=1)
    df = df.join(InternetService)
    OnlineSecurity = pd.get_dummies(df["OnlineSecurity"], prefix="OnlineSecurity")
    df.drop("OnlineSecurity", inplace=True, axis=1)
    df = df.join(OnlineSecurity)
    OnlineBackup = pd.get_dummies(df["OnlineBackup"], prefix="OnlineBackup")
    df.drop("OnlineBackup", inplace=True, axis=1)
    df = df.join(OnlineBackup)
    DeviceProtection = pd.get_dummies(df["DeviceProtection"], prefix="DeviceProtection")
    df.drop("DeviceProtection", inplace=True, axis=1)
    df = df.join(DeviceProtection)
    TechSupport = pd.get_dummies(df["TechSupport"], prefix="TechSupport")
    df.drop("TechSupport", inplace=True, axis=1)
    df = df.join(TechSupport)
    StreamingTV = pd.get_dummies(df["StreamingTV"], prefix="StreamingTV")
    df.drop("StreamingTV", inplace=True, axis=1)
    df = df.join(StreamingTV)
    StreamingMovies = pd.get_dummies(df["StreamingMovies"], prefix="StreamingMovies")
    df.drop("StreamingMovies", inplace=True, axis=1)
    df = df.join(StreamingMovies)
    Contract = pd.get_dummies(df["Contract"], prefix="Contract")
    df.drop("Contract", inplace=True, axis=1)
    df = df.join(Contract)
    PaymentMethod = pd.get_dummies(df["PaymentMethod"], prefix="PaymentMethod")
    df.drop("PaymentMethod", inplace=True, axis=1)
    df = df.join(PaymentMethod)
    y_df = df.pop("Churn")
    # x_df = df.drop("Churn", inplace=True, axis=1)

    return df, y_df


In [5]:
x, y = clean_data(df)

# Train test Split Dataset

In [7]:
# TODO: Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, stratify=y, random_state=42
)

train_dataset = pd.concat([x_train,pd.DataFrame(y_train)], axis=1)
test_dataset = pd.concat([x_test,pd.DataFrame(y_test)], axis=1)

# directory = 'train'
# if not os.path.exists(directory):
#     os.makedirs(directory)
    
# directory = 'test'
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Export data as csv
# train_dataset.to_csv("./train/train_data.csv", index=False)
# test_dataset.to_csv("./test/test_data.csv", index=False)

# # # Upload data to the datastore
# datastore = ws.get_default_datastore()
# datastore.upload(src_dir='./train', target_path = experiment_name)
# datastore.upload(src_dir='./test', target_path = experiment_name)
# print('Data uploaded to DataStore')

# csv_path_train = [(datastore, experiment_name+'/train_data.csv')]
# csv_path_test = [(datastore, experiment_name+'/test_data.csv')]

# train_data = Dataset.Tabular.from_delimited_files(path=csv_path_train)
# test_data = Dataset.Tabular.from_delimited_files(path=csv_path_test)

# display(train_data.to_pandas_dataframe().head())
# display(test_data.to_pandas_dataframe().head())


In [8]:
y_train.value_counts()

0    4139
1    1495
Name: Churn, dtype: int64

In [9]:
y_test.value_counts()

0    1035
1     374
Name: Churn, dtype: int64

## Cluster Provisioning

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


cluster_name = "cluster-vhcg"
# verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name = cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2', max_nodes = 4, idle_seconds_before_scaledown=120)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice, quniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--n_estimators': choice(10, 100, 200, 500, 700, 1000, 1500, 2000),
    '--max_features': uniform(0.5, 1.0),
    '--max_depth': choice(10, 20, 30, 40, 50, 60, 70, 80, 90, 100),
    '--min_samples_split': uniform(0.0001, 1.0),
    '--min_samples_leaf': uniform(0.00001, 0.5),
    # '--bootstrap': choice(True, False),
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=2, delay_evaluation=10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./', compute_target=cpu_cluster, entry_script='train_forest.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig( hyperparameter_sampling=ps, 
                                        primary_metric_name='AUC_weighted', 
                                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                        max_total_runs=25, 
                                        max_concurrent_runs=4, 
                                        max_duration_minutes=10080, 
                                        policy=policy, 
                                        estimator=est)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17
Web View: https://ml.azure.com/runs/HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-164149/workspaces/quick-starts-ws-164149&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-11-21T22:52:34.321773][API][INFO]Experiment created<END>\n""<START>[2021-11-21T22:52:35.126354][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-11-21T22:52:35.380565][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17
Web View: https://ml.azure.com/runs/HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17?wsid=/subscriptions/2c48c51c-bd47-40d4-abbe-fb8eabd19c8c/resourcegroups/aml-quickstarts-164149/workspaces/quick-starts-ws-164149&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17',
 'target': 'cluster-vhcg',
 'status': 'Completed',
 'startTimeUtc': '2021-11-21T22:52:34.057861Z',
 'endTimeUtc': '2021-11-21T23:08:24.007709Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "AUC_weighted", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '4ab4bce7-785c-4b04-a639-92bf99e35cf0',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1056-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.34.0',
  'score': '0.8493885018683947',
  'best_child_run_id': 'HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17_24',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg164149.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17/azureml-logs/hyperdrive.t

In [7]:
assert(hyperdrive_run.get_status() == 'Completed')

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best parameters: ', parameter_values)
print('Best Run Id: ', best_run.id)
print('\n AUC_weighted:', best_run_metrics['AUC_weighted'])


Best parameters:  ['--max_depth', '50', '--max_features', '0.8080872625286535', '--min_samples_leaf', '0.12168965575484306', '--min_samples_split', '0.21509880658124275', '--n_estimators', '200']
Best Run Id:  HD_ee1e5adf-007c-48a5-ab3e-0af5361faa17_24

 AUC_weighted: 0.8493885018683947


In [21]:
# #TODO: Save the best model
# best_model = best_run.register_model(model_name='udacity_project_model', model_path='./')
# best_model



#TODO: Save the best model
os.makedirs('./model', exist_ok=True)


best_run.download_file(name="./outputs/model.joblib", output_file_path="./model/model.joblib")

# Register
model = best_run.register_model(
    model_name='hyperdrive_model', 
    model_path='./outputs/model.joblib'
)
model

Model(workspace=Workspace.create(name='quick-starts-ws-164149', subscription_id='2c48c51c-bd47-40d4-abbe-fb8eabd19c8c', resource_group='aml-quickstarts-164149'), name=hyperdrive_model, id=hyperdrive_model:1, version=1, tags={}, properties={})

In [23]:
joblib.load("./model/model.joblib")

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
The sklearn.tree.tree module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.tree. Anything that cannot be imported from sklearn.tree is now part of the private API.
Trying to unpickle estimator DecisionTreeClassifier from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
Trying to unpickle estimator RandomForestClassifier from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as a

RandomForestClassifier(bootstrap=True, ccp_alpha=None, class_weight=None,
                       criterion='gini', max_depth=50,
                       max_features=0.8080872625286535, max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None,
                       min_samples_leaf=0.12168965575484306,
                       min_samples_split=0.21509880658124275,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
env = Environment.get(ws, "AzureML-Minimal").clone("udacity_ml")

for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [ ]:
env.save_to_directory("./env/", overwrite=True)

In [ ]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=2,
                                                enable_app_insights=True, auth_enabled=True) 

service_name = 'best-model-service'
service = Model.deploy(ws, service_name, [model], inference_config=inference_config, 
                       deployment_config=aci_config, overwrite=True)
service.wait_for_deployment(show_output = True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
x = iris.iloc[15, :-1].tolist()
y = iris.loc[15, "labels"]
print(x)
print(y)

In [ ]:
import json
input_payload = json.dumps([x])

output = service.run(input_payload)
print(f"Predicted: {output}\nExpected: {y}")

In [ ]:
print(service.get_logs())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()
compute_target.delete()